In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from skExSTraCS import ExSTraCS
from skrebate import ReliefF

In [ ]:
# Step 1: Importer le model de autoMLPipe et faire les graph par classe + classif report
cv_folder = "/home/meyer/code-project/histo-explore/mlpipe/results/myoxia/input/CVDatasets/"
all_cv = glob(cv_folder+"*")


In [ ]:
# Step 2: Entrainer Exstracs sur le dataset initial (concat de tout les test_df)
test_cv = glob(cv_folder+"*_Test.csv")
test_dfs = []
for i in test_cv:
    test_dfs.append(pd.read_csv(i))
original_dataset = pd.concat(test_dfs)
original_dataset.drop("id", axis=1, inplace=True)
Y = original_dataset["conclusion"].values
X = original_dataset.loc[:, original_dataset.columns != 'conclusion'].values
headers = original_dataset.loc[:, original_dataset.columns != 'conclusion'].columns.to_numpy()

# Get Feature Importance Scores to use as Expert Knowledge (see https://github.com/EpistasisLab/scikit-rebate/ for more details on skrebate package)
relieff = ReliefF()
relieff.fit(X, Y)
scores = relieff.feature_importances_

#Initialize and train model
learning_iterations = 200000
nu=1
N=500
rule_compaction="QRF"
random_state = 42
model = ExSTraCS(learning_iterations=learning_iterations, nu=nu, N=N,track_accuracy_while_fit=True,expert_knowledge=scores, rule_compaction=rule_compaction, random_state=random_state)

trainedModel = model.fit(X, Y)

trainedModel.export_iteration_tracking_data("/home/meyer/code-project/histo-explore/mlpipe/exstracs_train_full/iteration_results.csv")
trainedModel.export_final_rule_population(
    headers,
    "conclusion",
    filename="/home/meyer/code-project/histo-explore/mlpipe/exstracs_train_full/full_rules.csv",
    RCPopulation=False,
)
trainedModel.pickle_model("/home/meyer/code-project/histo-explore/mlpipe/exstracs_train_full/exstracs_model_preRC")
trainedModel.post_training_rule_compaction()
trainedModel.export_final_rule_population(
    headers,
    "conclusion",
    filename="/home/meyer/code-project/histo-explore/mlpipe/exstracs_train_full/compacted_rules.csv",
    RCPopulation=True,
)
trainedModel.pickle_model("/home/meyer/code-project/histo-explore/mlpipe/exstracs_train_full/exstracs_model_postRC")

In [ ]:
# Ecrire les règles, les règles dans un fichier et faire le graph de connaissance